# Mouse System Identification Models

In [1]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USERNAME']
dj.config['database.password'] = os.environ['DJ_PASSWORD']
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 200
        
name = 'vei'
os.environ["DJ_SCHEMA_NAME"] = f"metrics_{name}"
dj.config["nnfabrik.schema_name"] = os.environ["DJ_SCHEMA_NAME"]

In [2]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import hiplot as hip

from nnsysident.training.trainers import standard_trainer
from nnsysident.models.models import stacked2d_gamma, stacked2d_poisson
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.utility.data_helpers import extract_data_key
from nnsysident.utility.measures import get_model_performance

random_seed = 27121992
device = 'cuda'

Connecting konstantin@134.76.19.44:3306


In [28]:
from nnsysident.tables.bayesian import TrainedModelBayesian, ModelBayesian, DatasetBayesian
from nnfabrik.main import Model, Trainer, Dataset, Seed
from nnsysident.tables.experiments import TrainedModel

___

In [ ]:
hashes = tuple((Trainer & 'trainer_comment != "Bayesian optimization of Hyper params."').fetch("trainer_hash"))
TrainedModel.populate("trainer_hash in {}".format(hashes))

In [40]:
TrainedModel.progress("trainer_hash in {}".format(hashes))

TrainedModel         Completed 4 of 4 (100.0%)   2023-04-19 10:42:21


(0, 4)

In [47]:
aa = "test"

if aa == "test1":
    print("test1")
elif aa == "test2":
    print("test2")

In [49]:
TrainedModel().fetch()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,comment short description,score score,train_loss train_loss,validation_loss validation_loss,test_loss test_loss,train_correlation train_correlation,validation_correlation validation_correlation,test_correlation test_correlation,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nnsysident.models.models.stacked2d_gamma,ea7c8ee30c9f5ab0a632392c3a4b32c0,nnsysident.datasets.mouse_loaders.static_loaders,8e77e243f6a0377c9831e0f36d9093df,nnsysident.training.trainers.standard_trainer,69601593d387758e9ff6a5bf26dd6739,1,"Loss: GammaLoss, EarlyStopping: get_correlations.Best GammaModel from Hypersearch with EarlyStopping: get_correlations.",0.284335,-196627000.0,-22934300.0,-43280900.0,0.341546,0.284335,0.277896,=BLOB=,None,2023-04-19 12:19:13
nnsysident.models.models.stacked2d_gamma,ea7c8ee30c9f5ab0a632392c3a4b32c0,nnsysident.datasets.mouse_loaders.static_loaders,8e77e243f6a0377c9831e0f36d9093df,nnsysident.training.trainers.standard_trainer,93ced35f350b1f23b30e38c88257716b,1,"Loss: GammaLoss, EarlyStopping: get_loss.Best GammaModel from Hypersearch with EarlyStopping: get_correlations.",0.100197,-195615000.0,-22865500.0,-43156200.0,0.117735,0.100197,0.102089,=BLOB=,None,2023-04-19 12:24:16
nnsysident.models.models.stacked2d_poisson,eeea99f6abe39ec1226d71c55be03556,nnsysident.datasets.mouse_loaders.static_loaders,8e77e243f6a0377c9831e0f36d9093df,nnsysident.training.trainers.standard_trainer,69601593d387758e9ff6a5bf26dd6739,1,"Loss: GammaLoss, EarlyStopping: get_correlations.Best PoissonModel from old Hypersearch.",0.282825,17711000.0,2161320.0,4272440.0,0.370382,0.282825,0.273587,=BLOB=,None,2023-04-19 12:21:36
nnsysident.models.models.stacked2d_poisson,eeea99f6abe39ec1226d71c55be03556,nnsysident.datasets.mouse_loaders.static_loaders,8e77e243f6a0377c9831e0f36d9093df,nnsysident.training.trainers.standard_trainer,93ced35f350b1f23b30e38c88257716b,1,"Loss: GammaLoss, EarlyStopping: get_loss.Best PoissonModel from old Hypersearch.",0.284347,17705500.0,2156140.0,4252570.0,0.369391,0.284347,0.276673,=BLOB=,None,2023-04-19 12:26:31


## Data

In [31]:
paths = ['./data/static20457-5-9-preproc0']

data_key = extract_data_key(paths[0])

dataset_config = {'paths': paths,
                  'batch_size': 64,
                  'seed': random_seed,
                  'loader_outputs': ["images", "responses"],
                  'normalize': True,
                  'exclude': ["images"],
                  "cuda": True if device=="cuda" else False
                  }

dataloaders = static_loaders(**dataset_config)

## Model

In [ ]:
model_config_base = {"hidden_kern": 13,
                     "input_kern": 15,
                     "init_sigma": 0.4,
                     'init_mu_range': 0.55,
                     'gamma_input': 1.0,
                     'grid_mean_predictor': {'type': 'cortex',
                                              'input_dimensions': 2,
                                              'hidden_layers': 0,
                                              'hidden_features': 0,
                                              'final_tanh': False},
                     "feature_reg_weight": 0.78,
                     "readout_type": "MultipleGeneralizedFullGaussian2d",
                    }

# model_config_base['batch_norm_scale'] = False
# model_config_base['feature_reg_weight'] = 2.439
# model_config_base['independent_bn_bias'] = True

gamma_model = stacked2d_gamma(dataloaders, 
                            random_seed, 
                            **model_config_base)

poisson_model = stacked2d_poisson(dataloaders, random_seed, **model_config_base)

In [ ]:
not_matching_keys = gamma_model.load_state_dict(torch.load("lurz_core_poisson.tar"), strict=False)

not_matching_keys = list(key for key in not_matching_keys.missing_keys if key[:7] != "readout") + list(key for key in not_matching_keys.unexpected_keys if key[:7] != "readout")
print("{} not matching keys".format(len(not_matching_keys)))

In [ ]:
not_matching_keys = poisson_model.load_state_dict(torch.load("lurz_core_poisson.tar"), strict=False)

not_matching_keys = list(key for key in not_matching_keys.missing_keys if key[:7] != "readout") + list(key for key in not_matching_keys.unexpected_keys if key[:7] != "readout")
print("{} not matching keys".format(len(not_matching_keys)))

## Training

In [ ]:
trainer_config_base = {"track_training": False,
                       "device": device,
                       "detach_core": False,
                       "stop_function": "get_loss",
                       "maximize": False}

In [ ]:
gamma_score, gamma_output, gamma_state_dict = standard_trainer(gamma_model,
                                                                     dataloaders,
                                                                     random_seed,
                                                                     loss_function=gamma_model.loss_fn,
                                                                     **trainer_config_base)
# torch.save(gamma_state_dict, "mouseV1_gaussian_Gamma_statedict" + data_key + ".pt")

In [ ]:
gamma_performance = get_model_performance(gamma_model, dataloaders, gamma_model.loss_fn, device=device)

In [ ]:
poisson_score, poisson_output, poisson_state_dict = standard_trainer(poisson_model,
                                                                     dataloaders,
                                                                     random_seed,
                                                                     loss_function=poisson_model.loss_fn,
                                                                     **trainer_config_base)
# torch.save(poisson_state_dict, "mouseV1_gaussian_Poisson_statedict" + data_key + ".pt")

In [ ]:
poisson_performance = get_model_performance(poisson_model, dataloaders, poisson_model.loss_fn, device=device)

___